In [1]:
import os
import pandas as pd
import numpy as np
import sqlalchemy
import mysql.connector
import sys
import os.path
import math
import shutil
import getpass
from scipy import stats
from os import path
from mysql.connector import errorcode
from sqlalchemy import MetaData, create_engine
from sqlalchemy.exc import IntegrityError
from datetime import datetime, timedelta
from sqlalchemy.exc import ResourceClosedError, ProgrammingError

pd.options.mode.chained_assignment = None 
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# IEX packages
from iexfinance.refdata import *
from iexfinance.stocks import *
from iexfinance.iexdata import *
from iexfinance.stocks import get_historical_data
from iexfinance.account import get_metadata
from IPython.core.interactiveshell import InteractiveShell

import plotly.graph_objects as go
import plotly.express as px

InteractiveShell.ast_node_interactivity = "all"

In [2]:
BASE_DIR = 'C:\\Users\\pshar\\Dropbox\workspace\\HelloPython\\HistoricalMarketData\\TechnicalIndicators'
TABLE_EQUITIES_DATA = 'equities_historic_data'
DURATIONS = (14, 30, 90, 200) # Roughly for bi-weekly, monthly, quarterly, and 200 days running averages

In [3]:
def generateFilePath(symbol, date=None):
    if date is not None:
        strDate = datetime.strftime(date, '%Y%m%d')
        fileName = symbol.lower()+'_'+strDate+'.csv'
        filePath = os.path.join(BASE_DIR, fileName)
    else: 
        fileName = symbol.lower()+'.csv'
        filePath = os.path.join(BASE_DIR, fileName)
    if filePath is None:
            print('Could not find file for symbol:{}'.format(symbol))
    # print(filePath)
    return filePath, fileName

In [14]:
strDateTarget = '2012-01-03'
dtTarget = datetime.strptime(strDateTarget, '%Y-%m-%d')
# symbols = ('FB', 'MSFT', 'GOOGL', 'NFLX', 'AAPL', 'AMZN', 'WFC', 'TSLA', 'BAC', 'C', 'GS', 'JPM', 'MS', 'MRK', 'NKE')
symbols = ('TSLA', 'AMZN')
symbol = 'AAPL'
# for symbol in symbols:
filePath, _ = generateFilePath(symbol)
if (filePath is not None):
    try: 
        dfrm = pd.read_csv(filePath)
        dfrm.set_index('date', inplace=True)
    except FileNotFoundError as e:
        print('Exception reading input data for symbol {}.'.format(symbol))
        print(e)
        
dfrm.tail(10)

,symbol,close,volume,mean_200,stddev_200,pcntleStdDevs_200,pcntleVolume_200,pcntleClosing_200,oscillator_200,accu_dist_200,...,stddev_30,accu_dist_90,bollingerLower_90,bollingerUpper_90,mean_90,oscillator_90,pcntleClosing_90,pcntleStdDevs_90,pcntleVolume_90,stddev_90
date,,,,,,,,,,,,,,,,,,,,,
2020-07-31,AAPL,425.04,93573867,314.190360,42.234888,2.158273,97.841727,100.000000,100.000000,4.232526e+07,...,12.949684,3.524123e+07,285.008510,409.967297,347.487903,100.000000,100.000000,4.838710,100.000000,31.239697
2020-08-03,AAPL,435.75,77037847,315.089270,43.791821,2.919708,91.970803,100.000000,100.000000,4.276345e+07,...,16.491433,3.594519e+07,284.984494,414.590990,349.787742,100.000000,100.000000,6.451613,98.387097,32.401624
2020-08-04,AAPL,438.66,43198092,315.990146,45.046456,3.649635,64.233577,100.000000,100.000000,4.288017e+07,...,19.674383,3.604617e+07,284.851525,419.275572,352.063548,100.000000,100.000000,8.064516,80.645161,33.606012
2020-08-05,AAPL,440.25,30497988,316.877153,46.280327,4.379562,31.386861,100.000000,100.000000,4.285130e+07,...,22.076236,3.596414e+07,284.793354,423.837614,354.315484,100.000000,100.000000,9.677419,43.548387,34.761065
2020-08-06,AAPL,455.61,50607225,317.882464,47.599413,5.072464,76.811594,100.000000,100.000000,4.290750e+07,...,25.573808,3.631581e+07,283.867623,429.662377,356.765000,100.000000,100.000000,11.290323,91.935484,36.448689
2020-08-07,AAPL,444.45,49511403,318.793022,48.626465,5.755396,71.942446,99.280576,95.173845,3.880886e+07,...,27.098901,3.105664e+07,283.721686,434.141217,358.931452,92.457421,98.387097,12.903226,85.483871,37.604883
2020-08-10,AAPL,450.91,53100856,319.774420,50.081788,0.724638,79.710145,99.275362,97.967480,4.150701e+07,...,29.401513,3.450125e+07,283.243768,439.003006,361.123387,96.823466,98.387097,1.612903,95.161290,38.939810
2020-08-11,AAPL,437.50,46975594,320.631449,51.074517,1.449275,66.666667,96.376812,92.168310,3.661637e+07,...,29.647012,2.790107e+07,283.980858,442.333336,363.157097,87.760205,91.935484,3.225806,77.419355,39.588119
2020-08-12,AAPL,452.04,41486205,321.600507,52.284475,2.173913,57.246377,99.275362,98.456149,4.211041e+07,...,30.641936,3.502844e+07,284.474640,446.497296,365.485968,97.586207,98.387097,4.838710,69.354839,40.505664


In [16]:
dfrm.describe()

,close,volume,mean_200,stddev_200,pcntleStdDevs_200,pcntleVolume_200,pcntleClosing_200,oscillator_200,accu_dist_200,bollingerUpper_200,...,stddev_30,accu_dist_90,bollingerLower_90,bollingerUpper_90,mean_90,oscillator_90,pcntleClosing_90,pcntleStdDevs_90,pcntleVolume_90,stddev_90
count,4644.000000,4.644000e+03,4644.000000,4643.000000,4643.000000,4644.000000,4644.000000,4643.000000,4.643000e+03,4643.000000,...,4643.000000,4.643000e+03,4643.000000,4643.000000,4644.000000,4643.000000,4644.000000,4643.000000,4644.000000,4643.000000
mean,74.454001,1.136306e+08,69.373770,6.192112,57.545249,48.679055,70.507260,69.234850,3.925548e+07,81.772580,...,2.344807,2.823780e+07,63.402763,80.423561,71.898032,65.463952,65.621777,51.443253,49.161085,4.255199
std,79.139156,9.825176e+07,71.348368,7.414130,35.951274,30.235379,32.307721,31.437621,9.089704e+07,84.829383,...,3.187027,9.544983e+07,64.883198,84.955248,74.699440,32.661717,33.420514,36.534677,30.458774,5.776387
min,0.937000,9.870000e+06,1.051388,0.046136,0.709220,0.709220,0.709220,0.000000,-2.937347e+08,1.150458,...,0.017894,-3.377561e+08,0.814706,1.081309,1.025778,0.000000,1.562500,1.562500,1.562500,0.021348
25%,10.863750,4.235500e+07,9.722113,1.210019,23.741007,21.582734,50.000000,50.735111,8.646812e+05,12.040535,...,0.441914,-1.496758e+07,8.827304,11.913734,10.349082,40.336908,38.095238,15.873016,21.822917,0.734695
50%,49.570000,8.425910e+07,47.732601,3.641235,62.411348,48.175182,84.671533,80.883830,3.428544e+07,52.385316,...,1.365994,2.642005e+07,45.425161,51.860426,49.002618,75.409836,77.777778,48.837209,48.437500,2.444019
75%,112.665000,1.530900e+08,108.334745,7.885517,94.964029,75.716874,97.794118,95.451930,9.714132e+07,124.312508,...,2.949970,8.413069e+07,102.955927,120.843090,112.286537,95.373192,96.721311,91.999488,76.562500,5.425143
max,460.040000,8.432600e+08,322.596475,53.401671,100.000000,100.000000,100.000000,100.000000,2.737654e+08,429.399816,...,32.123775,3.048555e+08,285.008510,451.099304,367.913387,100.000000,100.000000,100.000000,100.000000,41.592958


In [15]:
# Using plotly.express
fig = px.line(dfrm, x=dfrm.index, y=['close', 'oscillator_30',  'mean_30'],  title='Time Series with Range Slider and Selectors')
# fig = px.line(dfrm, x=dfrm.index, y=['close', 'ema_14', 'ema_30'])
fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)
# fig.show()

## Strategies

### Buy / Sale Decisions Based on Bollinger Bands

In [8]:
duration = 90

In [9]:
dates_highs = [ date for date in dfrm.index if dfrm.loc[date, 'close'] > dfrm.loc[date, 'bollingerUpper_'+str(duration)] ]
dates_highs

['2002-04-17',
 '2002-10-30',
 '2002-10-31',
 '2002-11-01',
 '2002-11-04',
 '2002-11-05',
 '2002-11-06',
 '2003-05-05',
 '2003-05-06',
 '2003-05-07',
 '2003-05-08',
 '2003-05-09',
 '2003-05-12',
 '2003-05-13',
 '2003-05-14',
 '2003-05-15',
 '2003-05-16',
 '2003-05-19',
 '2003-05-27',
 '2003-08-29',
 '2003-09-02',
 '2003-09-03',
 '2003-09-04',
 '2003-09-12',
 '2003-10-10',
 '2003-10-13',
 '2003-10-14',
 '2003-10-15',
 '2004-03-04',
 '2004-03-05',
 '2004-03-08',
 '2004-03-09',
 '2004-03-10',
 '2004-03-11',
 '2004-03-12',
 '2004-03-29',
 '2004-03-30',
 '2004-04-05',
 '2004-04-15',
 '2004-06-07',
 '2004-06-08',
 '2004-06-09',
 '2004-06-10',
 '2004-06-14',
 '2004-06-15',
 '2004-06-16',
 '2004-06-17',
 '2004-06-18',
 '2004-06-21',
 '2004-06-22',
 '2004-06-23',
 '2004-06-24',
 '2004-06-25',
 '2004-08-26',
 '2004-08-27',
 '2004-08-31',
 '2004-09-01',
 '2004-09-02',
 '2004-09-03',
 '2004-09-07',
 '2004-09-08',
 '2004-09-09',
 '2004-09-10',
 '2004-09-17',
 '2004-09-20',
 '2004-09-21',
 '2004-09-

In [10]:
dates_lows = [ date for date in dfrm.index if dfrm.loc[date, 'close'] < dfrm.loc[date, 'bollingerUpper_'+str(duration)] ]
dates_lows

['2002-02-21',
 '2002-02-22',
 '2002-02-25',
 '2002-02-26',
 '2002-02-27',
 '2002-02-28',
 '2002-03-01',
 '2002-03-04',
 '2002-03-05',
 '2002-03-06',
 '2002-03-07',
 '2002-03-08',
 '2002-03-11',
 '2002-03-12',
 '2002-03-13',
 '2002-03-14',
 '2002-03-15',
 '2002-03-18',
 '2002-03-19',
 '2002-03-20',
 '2002-03-21',
 '2002-03-22',
 '2002-03-25',
 '2002-03-26',
 '2002-03-27',
 '2002-03-28',
 '2002-04-01',
 '2002-04-02',
 '2002-04-03',
 '2002-04-04',
 '2002-04-05',
 '2002-04-08',
 '2002-04-09',
 '2002-04-10',
 '2002-04-11',
 '2002-04-12',
 '2002-04-15',
 '2002-04-16',
 '2002-04-18',
 '2002-04-19',
 '2002-04-22',
 '2002-04-23',
 '2002-04-24',
 '2002-04-25',
 '2002-04-26',
 '2002-04-29',
 '2002-04-30',
 '2002-05-01',
 '2002-05-02',
 '2002-05-03',
 '2002-05-06',
 '2002-05-07',
 '2002-05-08',
 '2002-05-09',
 '2002-05-10',
 '2002-05-13',
 '2002-05-14',
 '2002-05-15',
 '2002-05-16',
 '2002-05-17',
 '2002-05-20',
 '2002-05-21',
 '2002-05-22',
 '2002-05-23',
 '2002-05-24',
 '2002-05-28',
 '2002-05-

### Buy / Sale Decisions Based on percentile of closing prices and volumes
- Sale needs to be made in two scenarios: a) Getting too high and need to cash out AND b) getting too low and need to limit losses. This is the second scenario where prices are too low but the sentinment to sale is high. 
- If percentile closing value is in lowest 10% but sale volume is in highest 90% then sale on the first day itself. Understand that a 90 day duration here works reasonably well. 
- However, if the same scenario (lowest 10% price, highest 90% volume) gets repeated say 3-4 times in two weeks then after that duration, send a signal for manual review for purchase. Dont buy it automatically because the company might actually be going down. Requires a human review of market condition of the company. 

In [11]:
dates_lows_for_buy_ops = [ date for date in dfrm.index if dfrm.loc[date, 'pcntleClosing_'+str(duration)] < 10 and dfrm.loc[date, 'pcntleVolume_'+str(duration)] > 90]
dates_lows_for_buy_ops

['2002-06-07',
 '2002-06-12',
 '2002-06-14',
 '2002-06-19',
 '2002-06-21',
 '2002-06-26',
 '2002-07-17',
 '2002-07-18',
 '2002-10-08',
 '2003-04-11',
 '2003-04-14',
 '2003-04-16',
 '2003-04-17',
 '2003-12-15',
 '2003-12-19',
 '2005-04-15',
 '2005-05-11',
 '2006-03-23',
 '2006-07-13',
 '2006-07-19',
 '2008-01-16',
 '2008-01-22',
 '2008-01-23',
 '2008-01-24',
 '2008-02-07',
 '2008-07-21',
 '2008-07-22',
 '2008-09-09',
 '2008-09-16',
 '2008-09-17',
 '2008-09-18',
 '2008-09-19',
 '2008-09-23',
 '2008-09-29',
 '2008-09-30',
 '2008-10-01',
 '2008-10-02',
 '2008-10-03',
 '2008-10-06',
 '2008-10-07',
 '2008-10-08',
 '2008-10-10',
 '2008-10-21',
 '2008-10-22',
 '2009-01-15',
 '2011-06-17',
 '2011-06-20',
 '2012-10-25',
 '2012-10-26',
 '2012-11-07',
 '2012-11-08',
 '2012-11-09',
 '2012-11-15',
 '2012-11-16',
 '2012-12-05',
 '2012-12-14',
 '2013-01-24',
 '2013-01-25',
 '2013-04-17',
 '2013-04-18',
 '2013-04-24',
 '2014-01-28',
 '2014-01-29',
 '2014-01-30',
 '2015-07-08',
 '2015-07-09',
 '2015-08-

In [12]:
dates_highs_for_sale_ops = [ date for date in dfrm.index if dfrm.loc[date, 'pcntleClosing_'+str(duration)] > 90 and dfrm.loc[date, 'pcntleVolume_'+str(duration)] > 90]
dates_highs_for_sale_ops

['2002-02-20',
 '2002-04-16',
 '2002-05-14',
 '2002-11-04',
 '2002-11-21',
 '2003-03-17',
 '2003-05-05',
 '2003-05-06',
 '2003-05-07',
 '2003-05-08',
 '2003-07-17',
 '2003-10-07',
 '2003-10-08',
 '2003-10-09',
 '2003-10-15',
 '2003-10-16',
 '2004-01-08',
 '2004-01-12',
 '2004-01-13',
 '2004-01-14',
 '2004-02-27',
 '2004-03-04',
 '2004-03-05',
 '2004-03-08',
 '2004-03-09',
 '2004-03-10',
 '2004-04-15',
 '2004-04-19',
 '2004-06-15',
 '2004-06-16',
 '2004-06-17',
 '2004-07-15',
 '2004-08-26',
 '2004-10-04',
 '2004-10-13',
 '2004-10-14',
 '2004-10-15',
 '2004-10-18',
 '2004-10-19',
 '2004-10-20',
 '2004-10-21',
 '2004-10-27',
 '2004-10-28',
 '2004-11-03',
 '2004-11-05',
 '2004-11-22',
 '2004-11-24',
 '2004-11-29',
 '2004-12-03',
 '2004-12-06',
 '2005-01-07',
 '2005-01-10',
 '2005-01-13',
 '2005-01-14',
 '2005-02-15',
 '2005-07-14',
 '2005-08-12',
 '2005-08-15',
 '2005-09-06',
 '2005-09-07',
 '2005-09-08',
 '2005-09-19',
 '2005-09-20',
 '2005-10-13',
 '2005-10-14',
 '2005-10-19',
 '2005-10-

### Buy / Sale based on Stochastic Oscillator

In [13]:
dates_lows_for_buy_ops_wo_stdev = [ date for date in dfrm.index if dfrm.loc[date, 'oscillator_'+str(duration)] < 25 and dfrm.loc[date, 'pcntleVolume_'+str(duration)] > 90 ]
dates_lows_for_buy_ops_w_stddev = [ date for date in dfrm.index if dfrm.loc[date, 'oscillator_'+str(duration)] < 25 and dfrm.loc[date, 'pcntleVolume_'+str(duration)] > 90 and dfrm.loc[date, 'pcntleStdDevs_'+str(duration)] > 90]
dates_lows_for_buy_ops_wo_stdev
dates_lows_for_buy_ops_w_stddev

['2002-02-21',
 '2002-02-27',
 '2002-06-07',
 '2002-06-12',
 '2002-06-14',
 '2002-06-19',
 '2002-06-21',
 '2002-06-26',
 '2002-07-16',
 '2002-07-17',
 '2002-07-18',
 '2002-10-08',
 '2002-10-17',
 '2003-01-15',
 '2003-01-16',
 '2003-01-27',
 '2003-01-30',
 '2003-04-11',
 '2003-04-14',
 '2003-04-16',
 '2003-04-17',
 '2003-11-24',
 '2003-12-15',
 '2003-12-19',
 '2005-04-14',
 '2005-04-15',
 '2005-05-11',
 '2006-02-10',
 '2006-03-23',
 '2006-03-29',
 '2006-06-08',
 '2006-07-13',
 '2006-07-19',
 '2007-08-16',
 '2008-01-16',
 '2008-01-22',
 '2008-01-23',
 '2008-01-24',
 '2008-02-07',
 '2008-07-21',
 '2008-07-22',
 '2008-09-09',
 '2008-09-16',
 '2008-09-17',
 '2008-09-18',
 '2008-09-23',
 '2008-09-29',
 '2008-09-30',
 '2008-10-01',
 '2008-10-02',
 '2008-10-03',
 '2008-10-06',
 '2008-10-07',
 '2008-10-08',
 '2008-10-10',
 '2008-10-14',
 '2008-10-16',
 '2008-10-21',
 '2008-10-22',
 '2009-01-15',
 '2010-01-29',
 '2010-02-05',
 '2011-03-16',
 '2011-06-17',
 '2011-06-20',
 '2012-10-25',
 '2012-10-

['2002-06-12',
 '2002-06-14',
 '2002-06-19',
 '2002-06-21',
 '2002-06-26',
 '2002-07-16',
 '2002-07-17',
 '2002-07-18',
 '2006-07-13',
 '2006-07-19',
 '2008-02-07',
 '2008-09-23',
 '2008-09-29',
 '2008-09-30',
 '2008-10-01',
 '2008-10-02',
 '2008-10-03',
 '2008-10-06',
 '2008-10-07',
 '2008-10-08',
 '2008-10-10',
 '2008-10-14',
 '2008-10-16',
 '2008-10-21',
 '2008-10-22',
 '2012-11-09',
 '2012-11-15',
 '2012-11-16',
 '2012-11-19',
 '2012-12-05',
 '2012-12-06',
 '2012-12-14',
 '2015-08-03',
 '2015-08-04',
 '2015-08-05',
 '2015-08-11',
 '2015-08-12',
 '2015-08-21',
 '2015-08-24',
 '2015-08-25',
 '2016-01-07',
 '2016-01-08',
 '2016-01-15',
 '2016-01-20',
 '2016-01-26',
 '2016-01-27',
 '2016-06-17',
 '2016-06-24',
 '2018-12-10',
 '2018-12-20',
 '2018-12-21',
 '2019-01-03',
 '2020-03-20',
 '2020-03-23']